# Info
API호출로 1차적으로 DB에 삽입된 데이터를 정제하여 2차적으로 DB에 삽입

DB삽입문 sql변수 코드에 {key}, 부분을 백틱없이 사용하여 replace구문으로 대체하였는데 백틱으로 할 경우 replace 구문이 필요 없을 것

In [1]:
import lostark as lok
import pandas as pd
from pprint import pprint
import traceback
import datetime

# ArmoryProfile

In [2]:
df = lok.get_df_raw_table('ArmoryProfile')

In [20]:
# lok.delete_table('stats')
# lok.delete_table('profile')

for i in range(len(df)):
    # 이미 입력된 값이면 다음 이름
#     result = lok.check_code_already_in("profile_table", df['characterCode'][i])
#     result2 = lok.check_code_already_in("stats_table", df['characterCode'][i])
#     if result[0] == 1:
#         print(f"characterCode {df['characterCode'][i]} is already in profile")
#     if result2[0] == 1:
#         print(f"characterCode {df['characterCode'][i]} is already in stats")
#     if (result[0] == 1) | (result2[0] == 1):
#         continue
    try:
        resp = lok.insert_profile_stats_table(df['characterCode'][i], df['data'][i])
    except:
        break
        
    if i % 10000 == 0:
        print("Record inserted successfully:", i)
        print("Current time:", datetime.datetime.now())

Record inserted successfully: 0
Current time: 2023-08-06 20:15:01.094440
Record inserted successfully: 10000
Current time: 2023-08-06 20:15:11.922883
Record inserted successfully: 20000
Current time: 2023-08-06 20:15:22.504270
Record inserted successfully: 30000
Current time: 2023-08-06 20:15:33.086657
Record inserted successfully: 40000
Current time: 2023-08-06 20:15:43.689801
Record inserted successfully: 50000
Current time: 2023-08-06 20:16:20.554011
Record inserted successfully: 60000
Current time: 2023-08-06 20:17:01.569199
Record inserted successfully: 70000
Current time: 2023-08-06 20:17:42.421543
Record inserted successfully: 80000
Current time: 2023-08-06 20:18:23.098422
Record inserted successfully: 90000
Current time: 2023-08-06 20:19:03.403688
Record inserted successfully: 100000
Current time: 2023-08-06 20:19:43.707778
Record inserted successfully: 110000
Current time: 2023-08-06 20:20:23.970987
Record inserted successfully: 120000
Current time: 2023-08-06 20:21:04.261542


# ArmoryAvatars

In [10]:
df = lok.get_df_raw_table('ArmoryAvatars')

In [146]:
# lok.delete_table('avatar')

# isInner가 True이다 == 2개 이상의 아바타를 끼고 있다? => 덧입기 중에는 효과가 발동되지 않음
for i in range(len(df)):
    try:
        lok.insert_avatar_table(df['characterCode'][i], df['data'][i])
    except:
        break
    
    if i % 10000 == 0:
        print("Record inserted successfully:", i)
        print("Current time:", datetime.datetime.now())

Record inserted successfully: 0
Current time: 2023-08-14 21:40:25.277601
Record inserted successfully: 10000
Current time: 2023-08-14 21:40:47.847592
Record inserted successfully: 20000
Current time: 2023-08-14 21:41:10.547654
Record inserted successfully: 30000
Current time: 2023-08-14 21:41:33.102259
Record inserted successfully: 40000
Current time: 2023-08-14 21:41:55.842922
Record inserted successfully: 50000
Current time: 2023-08-14 21:42:18.482173
Record inserted successfully: 70000
Current time: 2023-08-14 21:43:04.476203
Record inserted successfully: 80000
Current time: 2023-08-14 21:43:27.263333
Record inserted successfully: 90000
Current time: 2023-08-14 21:43:50.956813
Record inserted successfully: 100000
Current time: 2023-08-14 21:44:14.874882
Record inserted successfully: 110000
Current time: 2023-08-14 21:44:38.878519
Record inserted successfully: 120000
Current time: 2023-08-14 21:45:03.161621
Record inserted successfully: 130000
Current time: 2023-08-14 21:45:27.395436

# Armory Equipement

In [7]:
df = lok.get_df_raw_table('ArmoryEquipment')

In [115]:
# lok.delete_table('equipment')
# lok.delete_table('accessory')
# lok.delete_table('special_equipment')

for i in range(len(df)):
    try:
        lok.insert_equipment_accessory_sequipment_table(df['caharacterCode'][i], df['data'][i])
    except:
        break
    
    if i % 10000 == 0:
        print("Record inserted successfully:", i)
        print("Current time:", datetime.datetime.now())

Record inserted successfully: 0
Current time: 2023-08-15 17:25:19.789918
Record inserted successfully: 10000
Current time: 2023-08-15 17:27:44.180014
Record inserted successfully: 20000
Current time: 2023-08-15 17:30:02.107533
Record inserted successfully: 30000
Current time: 2023-08-15 17:32:53.064814
Record inserted successfully: 40000
Current time: 2023-08-15 17:35:50.809216
Record inserted successfully: 50000
Current time: 2023-08-15 17:38:46.208412
Record inserted successfully: 70000
Current time: 2023-08-15 17:45:11.472057
Record inserted successfully: 80000
Current time: 2023-08-15 17:48:25.357602
Record inserted successfully: 90000
Current time: 2023-08-15 17:51:39.572300
Record inserted successfully: 100000
Current time: 2023-08-15 17:54:53.098459
Record inserted successfully: 110000
Current time: 2023-08-15 17:58:04.465639
Record inserted successfully: 120000
Current time: 2023-08-15 18:01:18.286154
Record inserted successfully: 130000
Current time: 2023-08-15 18:04:34.789168

# ArmorySkills

In [12]:
# Armory Skills는 데이터가 너무 커서 10만개 씩 잘라서 처리
# df = lok.get_df_raw_table('ArmorySkills')

In [28]:
for idx in range(0, 580000, 5000):
    df = lok.get_df_raw_table('ArmorySkills', idx=idx)
    for i in range(len(df)):
        try:
            insert_skill_table(df['characterCode'][i], df['data'][i])
        except:
            break
        
    if idx % 10000 == 0:
        print("Record inserted successfully:", idx)
        print("Current time:", datetime.datetime.now())

Record inserted successfully: 0
Current time: 2023-08-16 14:23:10.898405
Record inserted successfully: 10000
Current time: 2023-08-16 14:23:40.182279
Record inserted successfully: 20000
Current time: 2023-08-16 14:24:20.203314
Record inserted successfully: 30000
Current time: 2023-08-16 14:25:09.797585
Record inserted successfully: 40000
Current time: 2023-08-16 14:26:09.423018
Record inserted successfully: 50000
Current time: 2023-08-16 14:27:19.584387
Record inserted successfully: 60000
Current time: 2023-08-16 14:28:40.931704
Record inserted successfully: 70000
Current time: 2023-08-16 14:30:13.460323
Record inserted successfully: 80000
Current time: 2023-08-16 14:31:55.897753
Record inserted successfully: 90000
Current time: 2023-08-16 14:33:48.942493
Record inserted successfully: 100000
Current time: 2023-08-16 14:35:51.871152
Record inserted successfully: 110000
Current time: 2023-08-16 14:38:06.206688
Record inserted successfully: 120000
Current time: 2023-08-16 14:40:30.716574


Traceback (most recent call last):
  File "C:\Users\user\AppData\Local\Temp\ipykernel_15700\3690718952.py", line 95, in <module>
    cursor.execute(sql, values)
  File "C:\Users\user\anaconda3\lib\site-packages\pymysql\cursors.py", line 156, in execute
    query = self.mogrify(query, args)
  File "C:\Users\user\anaconda3\lib\site-packages\pymysql\cursors.py", line 134, in mogrify
    query = query % self._escape_args(args, conn)
TypeError: not enough arguments for format string


Error inserting record: not enough arguments for format string
characterCode: 569923
{'characterCode': 569923,
 'skill10_backAttack': None,
 'skill10_cooltime': None,
 'skill10_counter': None,
 'skill10_headAttack': None,
 'skill10_level': None,
 'skill10_name': None,
 'skill10_partBreak': None,
 'skill10_runeGrade': None,
 'skill10_runeName': None,
 'skill10_stagger': None,
 'skill10_tripod1_name': None,
 'skill10_tripod1_point': None,
 'skill10_tripod2_name': None,
 'skill10_tripod2_point': None,
 'skill10_tripod3_name': None,
 'skill10_tripod3_point': None,
 'skill10_경직면역': None,
 'skill10_기력': None,
 'skill10_내공': None,
 'skill10_마나': None,
 'skill10_배터리': None,
 'skill10_버블': None,
 'skill10_상태이상면역': None,
 'skill10_영혼석': None,
 'skill10_충격': None,
 'skill10_피격면역': None,
 'skill11_backAttack': None,
 'skill11_cooltime': None,
 'skill11_counter': None,
 'skill11_headAttack': None,
 'skill11_level': None,
 'skill11_name': None,
 'skill11_partBreak': None,
 'skill11_runeGrade': None,

Traceback (most recent call last):
  File "C:\Users\user\AppData\Local\Temp\ipykernel_15700\3690718952.py", line 95, in <module>
    cursor.execute(sql, values)
  File "C:\Users\user\anaconda3\lib\site-packages\pymysql\cursors.py", line 156, in execute
    query = self.mogrify(query, args)
  File "C:\Users\user\anaconda3\lib\site-packages\pymysql\cursors.py", line 134, in mogrify
    query = query % self._escape_args(args, conn)
TypeError: not enough arguments for format string


Error inserting record: not enough arguments for format string
characterCode: 569923
{'characterCode': 569923,
 'skill10_backAttack': None,
 'skill10_cooltime': None,
 'skill10_counter': None,
 'skill10_headAttack': None,
 'skill10_level': None,
 'skill10_name': None,
 'skill10_partBreak': None,
 'skill10_runeGrade': None,
 'skill10_runeName': None,
 'skill10_stagger': None,
 'skill10_tripod1_name': None,
 'skill10_tripod1_point': None,
 'skill10_tripod2_name': None,
 'skill10_tripod2_point': None,
 'skill10_tripod3_name': None,
 'skill10_tripod3_point': None,
 'skill10_경직면역': None,
 'skill10_기력': None,
 'skill10_내공': None,
 'skill10_마나': None,
 'skill10_배터리': None,
 'skill10_버블': None,
 'skill10_상태이상면역': None,
 'skill10_영혼석': None,
 'skill10_충격': None,
 'skill10_피격면역': None,
 'skill11_backAttack': None,
 'skill11_cooltime': None,
 'skill11_counter': None,
 'skill11_headAttack': None,
 'skill11_level': None,
 'skill11_name': None,
 'skill11_partBreak': None,
 'skill11_runeGrade': None,

Traceback (most recent call last):
  File "C:\Users\user\AppData\Local\Temp\ipykernel_15700\3690718952.py", line 95, in <module>
    cursor.execute(sql, values)
  File "C:\Users\user\anaconda3\lib\site-packages\pymysql\cursors.py", line 156, in execute
    query = self.mogrify(query, args)
  File "C:\Users\user\anaconda3\lib\site-packages\pymysql\cursors.py", line 134, in mogrify
    query = query % self._escape_args(args, conn)
TypeError: not enough arguments for format string


Error inserting record: not enough arguments for format string
characterCode: 570021
{'characterCode': 570021,
 'skill10_backAttack': None,
 'skill10_cooltime': None,
 'skill10_counter': None,
 'skill10_headAttack': None,
 'skill10_level': None,
 'skill10_name': None,
 'skill10_partBreak': None,
 'skill10_runeGrade': None,
 'skill10_runeName': None,
 'skill10_stagger': None,
 'skill10_tripod1_name': None,
 'skill10_tripod1_point': None,
 'skill10_tripod2_name': None,
 'skill10_tripod2_point': None,
 'skill10_tripod3_name': None,
 'skill10_tripod3_point': None,
 'skill10_경직면역': None,
 'skill10_기력': None,
 'skill10_내공': None,
 'skill10_마나': None,
 'skill10_배터리': None,
 'skill10_버블': None,
 'skill10_상태이상면역': None,
 'skill10_영혼석': None,
 'skill10_충격': None,
 'skill10_피격면역': None,
 'skill11_backAttack': None,
 'skill11_cooltime': None,
 'skill11_counter': None,
 'skill11_headAttack': None,
 'skill11_level': None,
 'skill11_name': None,
 'skill11_partBreak': None,
 'skill11_runeGrade': None,

# ArmoryGem

In [42]:
df = lok.get_df_raw_table('ArmoryGem')

In [45]:
# lok.delete_table('gem')
for i in range(len(df)):
    try:
        lok.insert_gem_table(df['characterCode'][i], df['data'][i])
    except:
        break
        
    if i % 10000 == 0:
        print("Record inserted successfully:", i)
        print("Current time:", datetime.datetime.now())     

Record inserted successfully: 0
Current time: 2023-08-16 21:06:15.176556
Record inserted successfully: 10000
Current time: 2023-08-16 21:06:46.197095
Record inserted successfully: 20000
Current time: 2023-08-16 21:07:16.768904
Record inserted successfully: 40000
Current time: 2023-08-16 21:08:17.453408
Record inserted successfully: 80000
Current time: 2023-08-16 21:10:18.654642
Record inserted successfully: 90000
Current time: 2023-08-16 21:10:46.494521
Record inserted successfully: 110000
Current time: 2023-08-16 21:11:42.285524
Record inserted successfully: 170000
Current time: 2023-08-16 21:14:59.314970
Record inserted successfully: 180000
Current time: 2023-08-16 21:15:39.704013
Record inserted successfully: 190000
Current time: 2023-08-16 21:16:19.533670
Record inserted successfully: 200000
Current time: 2023-08-16 21:16:56.669385
Record inserted successfully: 220000
Current time: 2023-08-16 21:17:55.030937
Record inserted successfully: 240000
Current time: 2023-08-16 21:18:52.032

# ArmoryEngraving

In [51]:
df = lok.get_df_raw_table('ArmoryEngraving')

In [54]:
for i in range(len(df)):
    try:
        lok.insert_engraving_table(df['characterCode'][i], df['data'][i])
    except:
        break
        
    if i % 10000 == 0:
        print("Record inserted successfully:", i)
        print("Current time:", datetime.datetime.now())

Record inserted successfully: 10000
Current time: 2023-08-16 22:16:04.231308
Record inserted successfully: 20000
Current time: 2023-08-16 22:16:20.573490
Record inserted successfully: 30000
Current time: 2023-08-16 22:16:37.222291
Record inserted successfully: 40000
Current time: 2023-08-16 22:16:53.843362
Record inserted successfully: 50000
Current time: 2023-08-16 22:17:10.507112
Record inserted successfully: 70000
Current time: 2023-08-16 22:17:43.545820
Record inserted successfully: 80000
Current time: 2023-08-16 22:18:00.123615
Record inserted successfully: 90000
Current time: 2023-08-16 22:18:16.640653
Record inserted successfully: 100000
Current time: 2023-08-16 22:18:33.176364
Record inserted successfully: 110000
Current time: 2023-08-16 22:18:49.894277
Record inserted successfully: 120000
Current time: 2023-08-16 22:19:06.984356
Record inserted successfully: 130000
Current time: 2023-08-16 22:19:23.915046
Record inserted successfully: 140000
Current time: 2023-08-16 22:19:40.6

# ArmoryCard

In [64]:
df = lok.get_df_raw_table('ArmoryCard')

In [67]:
for i in range(len(df)):
    try:
        lok.insert_card_table(df['characterCode'][i], df['data'][i])
    except:
        break
        
    if i % 10000 == 0:
        print("Record inserted successfully:", i)
        print("Current time:", datetime.datetime.now())

Record inserted successfully: 10000
Current time: 2023-08-16 22:56:29.344932
Record inserted successfully: 20000
Current time: 2023-08-16 22:56:47.344126
Record inserted successfully: 30000
Current time: 2023-08-16 22:57:05.440819
Record inserted successfully: 40000
Current time: 2023-08-16 22:57:23.214064
Record inserted successfully: 50000
Current time: 2023-08-16 22:57:41.172750
Record inserted successfully: 70000
Current time: 2023-08-16 22:58:17.179918
Record inserted successfully: 80000
Current time: 2023-08-16 22:58:35.138190
Record inserted successfully: 90000
Current time: 2023-08-16 22:58:53.023520
Record inserted successfully: 100000
Current time: 2023-08-16 22:59:10.887655
Record inserted successfully: 110000
Current time: 2023-08-16 22:59:28.828620
Record inserted successfully: 120000
Current time: 2023-08-16 22:59:47.317814
Record inserted successfully: 130000
Current time: 2023-08-16 23:00:05.386259
Record inserted successfully: 140000
Current time: 2023-08-16 23:00:23.3

# Collectibles

In [11]:
df = lok.get_df_raw_table('Collectibles')

In [25]:
## ALTER TABLE
# ck = list(collectibles.keys())
# try:
#     for i in range(len(ck)-1):
#         sql = f"""ALTER TABLE `lostark`.`collectible_table` 
#         ADD COLUMN `{ck[i+1]}` TINYINT NULL AFTER `{ck[i]}`
#         """
#         cursor.execute(sql)
#         db.commit()
#     sql = """ALTER TABLE `lostark`.`collectible_table` 
#         CHANGE COLUMN `모코코_씨앗_대항해` `모코코_씨앗_대항해` SMALLINT NULL DEFAULT NULL ;
#         """
#     cursor.execute(sql)
#     db.commit()
# except Exception as e:
#     db.rollback()
#     traceback.print_exc()

In [39]:
for i in range(len(df)):
    try:
        lok.insert_collecible_table(df['characterCode'][i], df['data'][i])
    except:
        break
        
    if i % 10000 == 0:
        print("Record inserted successfully:", i)
        print("Current time:", datetime.datetime.now())

Record inserted successfully: 0
Current time: 2023-08-16 23:42:18.451933
Record inserted successfully: 10000
Current time: 2023-08-16 23:43:17.985630
Record inserted successfully: 20000
Current time: 2023-08-16 23:44:17.639542
Record inserted successfully: 30000
Current time: 2023-08-16 23:45:20.226789
Record inserted successfully: 40000
Current time: 2023-08-16 23:46:30.630858
Record inserted successfully: 50000
Current time: 2023-08-16 23:47:55.248389
Record inserted successfully: 60000
Current time: 2023-08-16 23:49:35.276435
Record inserted successfully: 70000
Current time: 2023-08-16 23:51:28.676467
Record inserted successfully: 80000
Current time: 2023-08-16 23:53:34.237699
Record inserted successfully: 90000
Current time: 2023-08-16 23:55:51.443786
Record inserted successfully: 100000
Current time: 2023-08-16 23:58:21.292862
Record inserted successfully: 110000
Current time: 2023-08-17 00:01:04.579045
Record inserted successfully: 120000
Current time: 2023-08-17 00:04:01.150155


# ColosseumInfo

In [11]:
df = lok.get_df_raw_table('ColosseumInfo')

In [58]:
## ALTERT TABLE
# ck = list(colosseums.keys())
# try:
#     for i in range(len(ck)-1):
#         sql = f"""ALTER TABLE lostark.colosseum_table 
#         ADD COLUMN `{ck[i+1]}` SMALLINT NULL AFTER `{ck[i]}`
#         """
#         cursor.execute(sql)
#         db.commit()
#     sql = """ALTER TABLE lostark.colosseum_table
#         CHANGE COLUMN `preSeason_Competitive_RankName` `preSeason_Competitive_RankName` VARCHAR(8) NULL DEFAULT NULL,
#         CHANGE COLUMN `preSeason_Competitive_Rank` `preSeason_Competitive_Rank` MEDIUMINT NULL DEFAULT NULL,
#         CHANGE COLUMN `exp` `exp` MEDIUMINT NULL DEFAULT NULL;
#         """
#     cursor.execute(sql)
#     db.commit()
#     for i in range(1, len(eval(df['data'][0])['Colosseums'])):
#         sql = f"""ALTER TABLE lostark.colosseum_table
#         CHANGE COLUMN `season{i}_Competitive_RankName` `season{i}_Competitive_RankName` VARCHAR(8) NULL DEFAULT NULL,
#         CHANGE COLUMN `season{i}_Competitive_Rank` `season{i}_Competitive_Rank` MEDIUMINT NULL DEFAULT NULL;
#         """
#         cursor.execute(sql)
#         db.commit()
# except Exception as e:
#     db.rollback()
#     traceback.print_exc()

In [59]:
for i in range(len(df)):
    try:
        lok.insert_colosseum_table(df['characterCode'][i], df['data'][i])
    except:
        break
        
    if i % 10000 == 0:
        print("Record inserted successfully:", i)
        print("Current time:", datetime.datetime.now())

Record inserted successfully: 0
Current time: 2023-08-17 09:19:52.569972
Record inserted successfully: 10000
Current time: 2023-08-17 09:20:17.940820
Record inserted successfully: 20000
Current time: 2023-08-17 09:20:44.302468
Record inserted successfully: 30000
Current time: 2023-08-17 09:21:10.897167
Record inserted successfully: 40000
Current time: 2023-08-17 09:21:36.734336
Record inserted successfully: 50000
Current time: 2023-08-17 09:22:02.426051
Record inserted successfully: 60000
Current time: 2023-08-17 09:22:28.427496
Record inserted successfully: 70000
Current time: 2023-08-17 09:22:55.319653
Record inserted successfully: 80000
Current time: 2023-08-17 09:23:21.811312
Record inserted successfully: 90000
Current time: 2023-08-17 09:23:48.077668
Record inserted successfully: 100000
Current time: 2023-08-17 09:24:13.298228
Record inserted successfully: 110000
Current time: 2023-08-17 09:24:39.456635
Record inserted successfully: 120000
Current time: 2023-08-17 09:25:05.571751


In [137]:
db.close()